In [ ]:
%matplotlib inline
import os
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
from rasterio.windows import Window
import sys
from shapely.geometry import mapping
sys.path.append(r'E:/gitlab/geoloc2/Detecciondeterrenos')
from codigos import Generar_txt
###path de yolo dentro de computadora
os.chdir(r'C:/Users/ASUS/Inteligencia_Artificial/yolov7')
from detect_Alberto_v4 import *
from scipy.ndimage import rotate as rotate_image
from shapely import geometry
import time
import datetime

In [ ]:
weights=r'C:\Users\ASUS\Inteligencia_Artificial\clasificador\ckpoint/best_mod11.pth'
num_classes=5
diciconario={0: 'casas', 1: 'en_construccion', 2: 'establecimiento', 3: 'multivivienda', 4: 'terreno_baldio'}
model_class=alexnet(weights=weights,num_classes=num_classes,idx_to_class=diciconario)

In [ ]:
# weights=r"C:\Users\ASUS\Inteligencia_Artificial\clasificador\ckpoint\best_modell11.pth"
# num_classes=6
# diciconario={0: 'carros', 1: 'casas', 2: 'en_construccion', 3: 'establecimiento', 4: 'multivivienda', 5: 'terreno_baldio'}
# model_class=alexnet(weights=weights,num_classes=num_classes,idx_to_class=diciconario)

In [ ]:
raster=r"C:\Users\ASUS\Desktop\temoaya.tif"
src=rasterio.open(raster)

In [ ]:
deteccion=r"C:\Users\ASUS\Desktop\presentacionyolov7\vero\vero\vero.shp"
deteccion_shp=gpd.read_file(deteccion)
deteccion_shp=deteccion_shp.dropna().reset_index(drop=True)
angulo_manzana=[]
for manzana in range(len(deteccion_shp)):
    proyecciones1=mapping(deteccion_shp['geometry'][manzana]).get('coordinates')
    angulos=[]
    d=[]
    poly=pd.DataFrame(proyecciones1[0])
    for point in range(1,len(poly)):
        d.append(((poly[1][point]-poly[1][point-1])**2+(poly[0][point]-poly[0][point-1])**2))
        angulos.append(math.atan(((poly[1][point]-poly[1][point-1])/(poly[0][point]-poly[0][point-1])))*180/math.pi)
    angulo_manzana.append(angulos[d.index(max(d))])
deteccion_shp['angulo']=angulo_manzana
deteccion_shp1=ampliar_shape(deteccion_shp,factor_ampliacion=1.25)

In [ ]:
import tqdm
import cv2
for i,polygo in tqdm.tqdm(enumerate(deteccion_shp1["geometry"]),total=len(deteccion_shp1)):
    det="a"
    try:
        shapes=[mapping(polygo)]
    except:
        continue
#     out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    four_images=[array[2],array[1],array[0]]
    stacked_images = np.stack(four_images, axis=-1)
    imagen_1=rotate_image(stacked_images,deteccion_shp["angulo"][i],reshape=False)
    clase,imagen=model_class.predict_image(imagen_1)
#     cv2.imshow("im",imagen)
#     cv2.waitKey()
#     cv2.destroyAllWindows()
    deteccion_shp.loc[i,"clase_n"]=clase

In [ ]:
len(deteccion_shp.dropna()[deteccion_shp["clase_dete"]==deteccion_shp["clase_n"]].drop(columns="centroid"))/len(deteccion_shp.dropna().drop(columns="centroid"))*100

In [ ]:
deteccion_shp.dropna()[deteccion_shp["clase_dete"]!=deteccion_shp["clase_n"]].drop(columns="centroid").to_file("D:/alexnet/a.shp")